# Customer Segmentation using RFM

İş Problemi : 

Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istemektedir.
Şirket, ortak davranışlar sergileyen müşteri segmentleri özelinde pazarlama çalışmaları yapmanın gelir artışı
sağlayacağını düşünmektedir.

Örneğin şirket için çok kazançlı olan müşterileri elde tutmak için farklı kampanyalar, yeni müşteriler için farklı
kampanyalar düzenlenmek istenmektedir.
    

### Veri Seti Hikayesi:

    Online Retail II https://archive.ics.uci.edu/ml/datasets/Online+Retail+II
isimli veri seti İngiltere merkezli online bir satış
mağazasının 01/12/2009 09/12/2011 tarihleri arasındaki satışlarını
içermektedir.
Bu şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır.
Şirketin müşterilerinin büyük çoğunluğu kurumsal müşterilerdir
    

### Veriyi Anlama (Data Understanding)

In [46]:
def check_df(dataframe, head=5):
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
import datetime as dt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

In [48]:
desired_width = 300
pd.set_option('display.width', desired_width)
pd.pandas.set_option('display.max_columns', None)

In [45]:
check_df(df)

##################### Head #####################
  Invoice StockCode                          Description  Quantity         InvoiceDate   Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00 2.55000  17850.00000  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00 3.39000  17850.00000  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00 2.75000  17850.00000  United Kingdom
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6 2010-12-01 08:26:00 3.39000  17850.00000  United Kingdom
4  536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6 2010-12-01 08:26:00 3.39000  17850.00000  United Kingdom
##################### Shape #####################
(541910, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quant

Verimiz 541910 satır 8 değişkenden oluşuyor. 
2 farklı sayısal değişken var gibi duruyor bunlar: UnitPrice,Quantity diğer sayısal gözüken değişkenlerin bir sayısal 
olarak bilgi taşımıyor sadece eşsiz değer mesela "Customer ID" gibi float gözüküyor ama sayısal olarak bir anlamı yok.

eşsiz değişkenler ise InvoiceNo,StockCode,CustomerID gibi duruyor.
birde tarih değişkeni var o da "InvoiceDate"

Eksik değişkenlerimiz var bunları şu anda işlediğimiz konu kapamına dahil etmediğimden şimdilik siliyorum.

Quantity min değer -80995
Price    min değer -11062
Eksi değer alan verilerimiz var bunlardan kurtulacağız bu - değerler iade edilen ürünleri temsil ediyor.

In [49]:
df.dropna(inplace=True)
print(df.isnull().sum())

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64


### Eşsiz ürün sayısı kaçtır?

In [50]:
df = pd.DataFrame(df)
df["StockCode"].nunique()

3684

### Hangi üründen kaçar tane vardır?

In [51]:
df[["Description","Quantity"]].groupby("Description").agg("sum")

,Quantity
Description,
4 PURPLE FLOCK DINNER CANDLES,140
50'S CHRISTMAS GIFT BAG LARGE,1883
DOLLY GIRL BEAKER,2391
I LOVE LONDON MINI BACKPACK,360
I LOVE LONDON MINI RUCKSACK,1
...,...
ZINC T-LIGHT HOLDER STARS SMALL,4850
ZINC TOP 2 DOOR WOODEN SHELF,5
ZINC WILLIE WINKIE CANDLE STICK,2595


### En çok sipariş edilen 5 ürünü çoktan aza doğru

In [53]:
df[["Description","Quantity"]].groupby("Description").agg("sum").sort_values("Quantity", ascending=False).head(5)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409


### Faturalardaki ‘C’ iptal edilen işlemleri göstermektedir. İptal edilen işlemleri veri setinden çıkartalım.

In [54]:
df = df[~df["Invoice"].str.contains("C", na=False)]
df["TotalPrice"] = df["Quantity"] * df["Price"]

### Recency (yenilik): Müşterinin son satın almasından bugüne kadar geçen süre
### Frequency (Sıklık): Toplam satın alma sayısı.
### Monetary (Parasal Değer): Müşterinin yaptığı toplam harcama.

In [56]:
today_date = dt.datetime(2011, 12, 11)

rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.columns = ["recency","frequency","monetary"]
rfm = rfm[rfm["monetary"] > 0]

In [57]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

In [58]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [59]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.00000,326,1,77183.60000,1,1,5,11,hibernating
12347.00000,3,7,4310.00000,5,5,5,55,champions
12348.00000,76,4,1797.24000,2,4,4,24,at_Risk
12349.00000,19,1,1757.55000,4,1,4,41,promising
12350.00000,311,1,334.40000,1,1,2,11,hibernating


In [60]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency         monetary      
                         mean count      mean count       mean count
segment                                                             
about_to_sleep       53.31250   352   1.16193   352  471.99437   352
at_Risk             153.78583   593   2.87858   593 1084.53530   593
cant_loose          132.96825    63   8.38095    63 2796.15587    63
champions             6.36177   633  12.41706   633 6857.96392   633
hibernating         217.60504  1071   1.10177  1071  488.64331  1071
loyal_customers      33.60806   819   6.47985   819 2864.24779   819
need_attention       52.42781   187   2.32620   187  897.62786   187
new_customers         7.42857    42   1.00000    42  388.21286    42
potential_loyalists  17.39876   484   2.01033   484 1041.22200   484
promising            23.51064    94   1.00000    94  294.00798    94

champions:

    RFM puanı 55
    recency_score = 5 , frequency_score =5
    * bu kullanıcılar en önemli kullanıcılarımız çünkü hem satın alma sıklıkları fazla hemde en son satın alma sureleri
        az. Bu müşterilerimizi kaybetmemek amacı ile iş gücü biraz daha buraya odaklanıp müşteriye ilgi gösterilmeli.
        
cant_loose:

    recency_score = 1,2 , frequency_score =5
    *bu müşterilerimizin satın alma sayıları yüksek fakat en son satın alma süresi düşük. Bu müşterilerimizi kaybetmemek
    için her müşteri özelinde bir inceleme olmalı. Belkide bu müşteriler aydan aya belli bir miktar alıp elindeki bitince
    tekrar almak için dönüyor olabilir. bunları ayrıştırmamız gerek. ve eğer uzun süüredir bizimle çalışan bir müşteri ise
    ona özel kampanya yapılabilinir.
new_customers:

    recency_score = 5 , frequency_score = 1
    Bu kullanıcılar yeni geldiğinden onları burada tutan şeyin ne olacağını bulmak gerekir. ve bunlarada önem gösterilmesi
    lazım. ona özel reklam mesajları atabilinir. kendini özel hissettirecel mail de atılabilinir. 
